# Telecom X — Parte 2 · Predicción de Churn (Versión C)

**Diferencias**: feature engineering alternativo, **XGBoost**, **Lift/Gain**, umbral por **ganancia** y **Youden J**, calibración comparada por **Brier** y **PR-AUC**.

In [ ]:
# === Dependencias (solo Colab) ===
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False
if IN_COLAB:
    !pip -q install xgboost imbalanced-learn >/dev/null


In [ ]:
# === Imports y setup ===
import os, json, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (roc_auc_score, average_precision_score, precision_recall_curve, roc_curve,
                             confusion_matrix, classification_report, brier_score_loss)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_selection import mutual_info_classif
warnings.filterwarnings('ignore')
RANDOM_STATE = 202
np.random.seed(RANDOM_STATE)
USE_BALANCING = False
BALANCING_METHOD = 'ros'


In [ ]:
# === Parámetros de negocio y paths ===
CSV_PATH = '/content/df_limpo.csv'
VALUE_RETAIN = 110.0
COST_CONTACT = 6.0
TOP_N = 400
os.makedirs('/content/reports', exist_ok=True)
os.makedirs('/content/data/interim', exist_ok=True)


## 1) Carga + Feature Engineering (alternativo)

In [ ]:
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f'No se encontró df_limpo.csv en {CSV_PATH}')
df = pd.read_csv(CSV_PATH)
print('Shape df:', df.shape)
display(df.head(3))

assert 'Churn' in df.columns, "El CSV tratado debe incluir la columna 'Churn'"
y_raw = df['Churn'].astype('string').str.strip().str.lower()
mask = y_raw.isin(['yes','no'])
dfm = df.loc[mask].copy()
dfm['Churn_bin'] = y_raw.loc[mask].map({'yes':1,'no':0}).astype(int)

# Feature engineering alternativo
def to_str(s):
    return s.astype('string').str.strip().str.lower()

# Indicadores de contrato e internet
if 'account.Contract' in dfm.columns:
    c = to_str(dfm['account.Contract'])
    dfm['contract_m2m'] = (c=='month-to-month').astype(int)
else:
    dfm['contract_m2m'] = 0

if 'internet.InternetService' in dfm.columns:
    i = to_str(dfm['internet.InternetService'])
    dfm['is_fiber'] = i.str.contains('fiber', na=False).astype(int)
    dfm['is_dsl'] = (i=='dsl').astype(int)
else:
    dfm['is_fiber'] = 0; dfm['is_dsl'] = 0

# Auto-pay (pagos automáticos) desde PaymentMethod
if 'account.PaymentMethod' in dfm.columns:
    p = to_str(dfm['account.PaymentMethod'])
    dfm['auto_pay'] = p.str.contains('automatic', na=False).astype(int)
else:
    dfm['auto_pay'] = 0

# Tenure bucket alternativo
tenure_col = None
for c in dfm.columns:
    if 'tenure' in c.lower():
        tenure_col = c; break
if tenure_col:
    dfm['_tenure'] = pd.to_numeric(dfm[tenure_col], errors='coerce')
    dfm['tenure_bucket'] = pd.cut(dfm['_tenure'], bins=[-1,1,3,6,12,24,36,48,60,1e9],
                                  labels=['0-1','2-3','4-6','7-12','13-24','25-36','37-48','49-60','60+'])
else:
    dfm['tenure_bucket'] = 'unk'

# Cargas: convertir a num y generar charge_density
mc_col, tc_col = None, None
for c in dfm.columns:
    lc = c.lower()
    if 'monthly' in lc and 'charge' in lc: mc_col = c
    if 'total' in lc and 'charge' in lc: tc_col = c
if mc_col:
    dfm['_mc'] = pd.to_numeric(dfm[mc_col], errors='coerce')
if tc_col:
    dfm['_tc'] = pd.to_numeric(dfm[tc_col], errors='coerce')
if mc_col and tenure_col:
    dfm['charge_density'] = dfm['_mc'] / (dfm['_tenure']+1)
else:
    dfm['charge_density'] = np.nan

# Definir X/y
X = dfm.drop(columns=['Churn','Churn_bin','customerID'], errors='ignore')
y = dfm['Churn_bin'].to_numpy()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()
print('num_cols:', len(num_cols), '| cat_cols:', len(cat_cols))


## 2) Preprocesamiento + Split

In [ ]:
try:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)  # sklearn >=1.2
except TypeError:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)         # sklearn <1.2

num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', ohe)
])
trs = []
if len(num_cols)>0: trs.append(('num', num_pipe, num_cols))
if len(cat_cols)>0: trs.append(('cat', cat_pipe, cat_cols))
pre = ColumnTransformer(trs)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)
print('Train/Test:', X_train.shape, X_test.shape)


## 3) Modelos + GridSearch (incluye XGBoost)

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

models = {}
models['logreg'] = (LogisticRegression(max_iter=2000, class_weight='balanced'),
                    {'model__C':[0.1,1,3,10]})
models['gb'] = (GradientBoostingClassifier(random_state=RANDOM_STATE),
                {'model__n_estimators':[200,400], 'model__learning_rate':[0.05,0.1], 'model__max_depth':[2,3]})

# XGBoost (si está disponible)
try:
    from xgboost import XGBClassifier
    xgb = XGBClassifier(random_state=RANDOM_STATE, n_estimators=500, subsample=0.9, colsample_bytree=0.9,
                        eval_metric='logloss', tree_method='hist', n_jobs=-1)
    models['xgb'] = (xgb, {
        'model__max_depth':[3,5,7],
        'model__learning_rate':[0.03, 0.1],
        'model__min_child_weight':[1,3]
    })
except Exception as e:
    print('XGBoost no disponible, se omite:', e)

best_est, best_name, best_score = None, None, -np.inf
scores = {}
for name, (est, grid) in models.items():
    pipe = Pipeline([('pre', pre), ('model', est)])
    gs = GridSearchCV(pipe, grid, scoring='average_precision', cv=cv, n_jobs=-1)
    gs.fit(X_train, y_train)
    scores[name] = gs.best_score_
    if gs.best_score_ > best_score:
        best_est, best_name, best_score = gs.best_estimator_, name, gs.best_score_

print('PR-AUC (CV) por modelo:')
for k,v in scores.items():
    print(f'{k:8s} -> {v:.4f}')
print(f'Mejor: {best_name} (PR-AUC CV={best_score:.4f})')


## 4) Calibración y métricas (Brier + PR-AUC)

In [ ]:
pre_fitted = best_est.named_steps['pre'].fit(X_train, y_train)
Xtr = pre_fitted.transform(X_train)
Xte = pre_fitted.transform(X_test)
base = best_est.named_steps['model']

def calibrate_compare(base, Xtr, ytr, Xte, yte):
    cand = {}
    try:
        iso = CalibratedClassifierCV(base, method='isotonic', cv=3).fit(Xtr, ytr)
        p = iso.predict_proba(Xte)[:,1]; cand['isotonic'] = (p, brier_score_loss(yte, p), average_precision_score(yte, p))
    except Exception:
        pass
    try:
        pl = CalibratedClassifierCV(base, method='sigmoid', cv=3).fit(Xtr, ytr)
        p = pl.predict_proba(Xte)[:,1]; cand['platt'] = (p, brier_score_loss(yte, p), average_precision_score(yte, p))
    except Exception:
        pass
    if not cand:
        if hasattr(base, 'predict_proba'):
            p = base.fit(Xtr, ytr).predict_proba(Xte)[:,1]
        else:
            s = base.fit(Xtr, ytr).decision_function(Xte)
            s = (s-s.min())/(s.max()-s.min()+1e-9); p = s
        return p, 'none', brier_score_loss(yte, p), average_precision_score(yte, p)
    # criterio mixto: menor Brier y mayor PR-AUC (ponderación simple)
    best = None; best_key = None
    for k,(pp, bs, prauc) in cand.items():
        score = -bs + prauc  # mayor es mejor
        if best is None or score > best[0]:
            best = (score, pp, bs, prauc, k)
    _, pp, bs, prauc, k = best
    return pp, k, bs, prauc

y_score, cal_used, brier, pr_auc = calibrate_compare(base, Xtr, y_train, Xte, y_test)
roc = roc_auc_score(y_test, y_score)
print(f'Calibración: {cal_used} | Brier: {brier:.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr_auc:.4f}')

prec, rec, _ = precision_recall_curve(y_test, y_score)
fpr, tpr, _ = roc_curve(y_test, y_score)
plt.figure(figsize=(6,4)); plt.plot(rec, prec); plt.xlabel('Recall'); plt.ylabel('Precision'); plt.title('PR curve'); plt.show()
plt.figure(figsize=(6,4)); plt.plot(fpr, tpr); plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC curve'); plt.show()


## 5) Lift & Cumulative Gain

In [ ]:
def lift_gain(y_true, y_score, buckets=10):
    dfm = pd.DataFrame({'y':y_true, 'p':y_score})
    dfm = dfm.sort_values('p', ascending=False).reset_index(drop=True)
    dfm['bucket'] = pd.qcut(dfm.index+1, q=buckets, labels=False)
    grp = dfm.groupby('bucket').agg({'y':['sum','count']})
    grp.columns = ['positives','n']
    grp['cum_positives'] = grp['positives'].cumsum()
    grp['cum_n'] = grp['n'].cumsum()
    base_rate = dfm['y'].mean()
    grp['gain'] = grp['cum_positives']/grp['positives'].sum()
    grp['lift'] = (grp['positives']/grp['n'])/base_rate
    return grp, base_rate

lg, base = lift_gain(y_test, y_score, buckets=10)
plt.figure(figsize=(6,4)); plt.plot((lg['cum_n']/lg['n'].sum()), lg['gain']); plt.xlabel('Porción de clientes'); plt.ylabel('Cumulative Gain'); plt.title('Curva de Ganancia Acumulada'); plt.show()
plt.figure(figsize=(6,4)); plt.bar(range(1,11), lg['lift']); plt.xlabel('Decil'); plt.ylabel('Lift'); plt.title('Lift por decil'); plt.show()


## 6) Umbral de negocio + Youden J

In [ ]:
def business_profit(y_true, p, thr, value=VALUE_RETAIN, cost=COST_CONTACT):
    y_pred = (p>=thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tp*value - (tp+fp)*cost

thr_grid = np.linspace(0.005, 0.5, 300)
profits = np.array([business_profit(y_test, y_score, t) for t in thr_grid])
thr_best = float(thr_grid[int(np.argmax(profits))]); profit_best = float(np.max(profits))

# Youden J (sens + esp - 1)
fpr, tpr, thr = roc_curve(y_test, y_score)
youden_idx = int(np.argmax(tpr - fpr))
thr_j = float(thr[youden_idx]) if youden_idx < len(thr) else thr_best

print(f'Threshold negocio (ganancia): {thr_best:.3f} | Ganancia: {profit_best:.2f}')
print(f'Threshold Youden J: {thr_j:.3f}')
plt.figure(figsize=(6,4)); plt.plot(thr_grid, profits); plt.axvline(thr_best, ls='--'); plt.xlabel('Threshold'); plt.ylabel('Profit'); plt.title('Curva de ganancia'); plt.show()

for t in [thr_best, thr_j]:
    y_pred = (y_score>=t).astype(int)
    print(f'--- Umbral {t:.3f} ---')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=3))


## 7) Artefactos

In [ ]:
metrics = {
  'best_model': best_name,
  'cv_best_pr_auc': float(best_score),
  'test_roc_auc': float(roc_auc_score(y_test, y_score)),
  'test_pr_auc': float(average_precision_score(y_test, y_score)),
  'brier_score': float(brier),
  'calibration': cal_used,
  'thr_profit': float(thr_best),
  'profit_best': float(profit_best),
  'thr_youden': float(thr_j),
  'value_retain': float(VALUE_RETAIN),
  'cost_contact': float(COST_CONTACT)
}
with open('/content/reports/metrics_p2_alt.json','w') as f:
    json.dump(metrics, f, indent=2)
print('Guardado metrics_p2_alt.json')

lines = []
lines.append('# Informe de Resultados — Telecom X · Parte 2 (Versión C)')
lines.append('')
lines.append('## 1. Datos y preparación')
lines.append(f'- Filas: {df.shape[0]} | Columnas: {df.shape[1]}')
lines.append('')
lines.append('## 2. Modelo y calibración')
lines.append(f'- Mejor por CV (PR-AUC): **{best_name}** → {best_score:.4f}')
lines.append(f'- Test: ROC-AUC={roc_auc_score(y_test, y_score):.4f} | PR-AUC={average_precision_score(y_test, y_score):.4f} | Brier={brier:.4f} | cal={cal_used}')
lines.append('')
lines.append('## 3. Umbral de negocio')
lines.append(f'- Ganancia máxima: thr={thr_best:.3f} → {profit_best:.2f} (VALUE_RETAIN={VALUE_RETAIN}, COST_CONTACT={COST_CONTACT})')
lines.append(f'- Youden J: thr={thr_j:.3f}')
with open('/content/reports/README_REPORT_P2_alt.md','w', encoding='utf-8') as f:
    f.write('\n'.join(lines))
print('Guardado README_REPORT_P2_alt.md')

# Export Top-N
salida = dfm.loc[X_test.index].copy()
salida['churn_score'] = y_score
salida['flag_churn_risk'] = (y_score >= thr_best).astype(int)
topN = salida.sort_values('churn_score', ascending=False).head(TOP_N)
top_path = '/content/reports/clientes_en_riesgo_topN_p2_alt.csv'
topN.to_csv(top_path, index=False)
print('Exportado', top_path)
topN.head(5)
